In [821]:
from os import path

full_input = dict()
with open(path.join(globals()['_dh'][0], "input.txt")) as f:
    lines = [x.strip() for x in f.readlines() if len(x.strip()) > 0]
    draws = [int(x) for x in lines[0].split(',')]
    lines = lines[1:]
    boards = [lines[i:i + 5] for i in range(0, len(lines), 5)]

    full_input['draws'] = draws
    full_input['boards'] = boards

test_input = dict()
with open(path.join(globals()['_dh'][0], "test.txt")) as f:
    lines = [x.strip() for x in f.readlines() if len(x.strip()) > 0]
    draws = [int(x) for x in lines[0].split(',')]
    lines = lines[1:]
    boards = [lines[i:i + 5] for i in range(0, len(lines), 5)]

    test_input['draws'] = draws
    test_input['boards'] = boards

In [822]:
class Cell:
    _marked = False
    value = None

    def __init__(self, value):
        self.value = value

    def set_marked(self):
        self._marked = True

    def is_marked(self):
        return self._marked


class Board:
    N = 5

    def __init__(self, board_list):
        self.rows = dict()
        self.columns = dict()
        self.cells = dict()
        rows = [[int(x.strip()) for x in row.split()] for row in board_list]

        self.cells = {x: Cell(x) for row in rows for x in row}

        for row in rows:
            key = ' '.join([str(x) for x in row])
            value = [self.cells[x] for x in row]
            self.rows[key] = value

        for i in range(self.N):
            column = []
            for row in self.rows.keys():
                column.append(row.split()[i])
            self.columns[' '.join(column)] = [self.cells[int(x)] for x in column]

    def mark_cell(self, value):
        if value in self.cells: self.cells[value].set_marked()

    def _has_winning_row(self):
        for row, cells in self.rows.items():
            marked_cells = [1 if c.is_marked() else 0 for c in cells]
            if sum(marked_cells) == self.N: return True

        return False

    def _has_winning_column(self):
        for column, cells in self.columns.items():
            marked_cells = [1 if c.is_marked() else 0 for c in cells]
            if sum(marked_cells) == self.N: return True

        return False

    def is_winning_board(self):
        return self._has_winning_column() or self._has_winning_row()

    def print_board(self):
        rows = []
        for row in self.rows.keys():
            split = row.split()
            values = []
            for x in split:
                value = '*' if self.cells[int(x)].is_marked() else x
                values.append(value)
            rows.append(' '.join(values))
        print('\n'.join(rows))
        print()

    def get_board_score(self):
        return sum([c.value for c in list(filter(lambda x: not x.is_marked(), self.cells.values()))])

In [823]:
def part1(draws, boards):
    board_objects = [Board(board) for board in boards]
    for n in draws:
        for board in board_objects:
            board.mark_cell(n)
        winning_boards = list(filter(lambda x: x.is_winning_board(), board_objects))
        if len(winning_boards) > 0:
            return n * winning_boards[0].get_board_score()

In [824]:
def part2(draws, boards):
    board_objects = {Board(board): False for board in boards}
    for n in draws:
        boards_to_mark = [board for board, is_winner in board_objects.items() if not is_winner]
        for board in boards_to_mark:
            board.mark_cell(n)
            if board.is_winning_board(): board_objects[board] = True
        if len(boards_to_mark) == 1:
            return n * boards_to_mark[0].get_board_score()

In [825]:
draws = full_input['draws']
boards = full_input['boards']

print(f"Solution to part 1 is {part1(draws, boards)}")
print(f"Solution to part 2 is {part2(draws, boards)}")

Solution to part 1 is 6592
Solution to part 2 is 31755
